In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# %matplotlib inline # plot in cell
from sklearn import metrics
import pandas as pd
from scipy import signal
from scipy import stats

import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# for svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

In [98]:
non_fall = pd.read_excel(r'F:\Program\OneDrive\Máy tính\fall-nonfall\non-fall.xlsx')
non_fall.columns

fall = pd.read_excel(r'F:\Program\OneDrive\Máy tính\fall-nonfall\fall.xlsx')
fall.columns

Index(['Acc_X', 'Acc_Y', 'Acc_Z'], dtype='object')

Index(['Acc_X', 'Acc_Y', 'Acc_Z'], dtype='object')

In [99]:
#check number of NaN
non_fall.isnull().sum().sum()
fall.isnull().sum().sum()

0

0

In [100]:
len(non_fall)
len(fall)

601030

675039

In [113]:
frame = 110
stride = 110
ratio = 0.6

non_fall_train = [non_fall[i:i+frame] for i in range(0, int(len(non_fall)*ratio), stride)] 
non_fall_test = [non_fall[i:i+frame] for i in range(int(len(non_fall)*ratio), len(non_fall), stride)
                                                    if i+frame<=len(non_fall)]

fall_train = [fall[i:i+frame] for i in range(0, int(len(fall)*ratio), stride)] 
fall_test = [fall[i:i+frame] for i in range(int(len(fall)*ratio), len(fall), stride) 
                                            if i+frame<=len(fall)]

In [114]:
train_data = []
train_label = []

test_data = []
test_label = []

for acts in non_fall_train:
    train_data.append(acts)
    train_label.append(0)
    
for acts in fall_train:
    train_data.append(acts)
    train_label.append(1)


print('train-data length: ', len(train_data))
print('train-label length: ', len(train_label))
#print(train_label)
      
# For TEST

for acts in non_fall_test:
    test_data.append(acts)
    test_label.append(0)
    
for acts in fall_test:
    test_data.append(acts)
    test_label.append(1)
    

train-data length:  6962
train-label length:  6962


In [115]:
# def diffState(x,y,z):
#     result = 0
#     for i,j,k in zip(x,y,z):
#         summ = pow((i-np.mean(x)),2)+pow((j-np.mean(y)),2)+pow((k-np.mean(z)),2)
#         result = result + summ
#     return np.sqrt(result)

# from scipy.stats import kurtosis, skew
# def averageLength(x,y,z):
#     summ = 0
#     for i,j,k in zip(x,y,z):
#         summ += i*i + j*j + k*k
#     summ = np.sqrt(summ)
#     return summ

def featuresFromBuffer(at):
    feat = np.zeros(15)   
    
    x = np.array(at.iloc[:,0], dtype=np.float64)   
    y = np.array(at.iloc[:,1], dtype=np.float64)   
    z = np.array(at.iloc[:,2], dtype=np.float64)  
    
    means = [np.mean(i) for i in [x, y, z]]
    feat[0:3] = means 
    
    rms = [np.sqrt(np.mean(i**2)) for i in [x, y, z]]
    feat[3:6] = rms

    # Standard deviation
    std = [np.std(i) for i in [x, y, z]]
    feat[6:9] = std

    # Median
    med = [np.median(i) for i in [x, y, z]]
    feat[9:12] = med
    
    # Range
    Range = [np.amax(i)-np.amin(i) for i in [x, y, z]]    
    feat[12:15] = Range
    
#     #max
#     maxValue = [np.amax(i) for i in [x,y,z]]
#     feat[15:18] = maxValue

#     minValue = [np.amin(i) for i in [x,y,z]]
#     feat[18:21] = minValue
    
#     #skew
#     skewValue = [skew(i) for i in [x,y,z]]
#     feat[21:24] = skewValue
    
#     #kurtosis
#     kurtosisValue = [kurtosis(i) for i in [x,y,z]]
#     feat[24:27] = kurtosisValue
    
#     #slope 
#     slope = np.sqrt(pow(np.amax(x)-np.amin(x),2)+pow(np.amax(y)-np.amin(y),2)+pow(np.amax(z)-np.amin(z),2))
#     feat[27] = slope
    
#     feat[28] = averageLength(x,y,z)

# #abs
#     meanAbs = [abs(np.mean(i)) for i in [x,y,z]]
#     feat[27:30] = meanAbs
    
#     medianAbs = [abs(np.median(i)) for i in [x, y, z]]
#     feat[30:33] = medianAbs
    
#     stdAbs = [abs(np.std(i)) for i in [x, y, z]]
#     feat[33:36] = stdAbs
    
#     minAbs = [abs(np.amin(i)) for i in [x,y,z]]
#     feat[36:39] = minAbs
    
#     maxAbs = [abs(np.amax(i)) for i in [x,y,z]]
#     feat[39:42] = maxAbs
    
#     skewAbs = [abs(skew(i)) for i in [x,y,z]]
#     feat[42:45] = skewAbs
    
#     kurtosisAbs = [abs(kurtosis(i)) for i in [x,y,z]]
#     feat[45:48] = kurtosisAbs
    
    
#     #tilt angle 
#     #mean 
#     feat[29] = np.arcsin(np.mean(y)/averageLength(x,y,z))
#     #std
#     feat[30] = np.arcsin(np.std(y)/averageLength(x,y,z))
#     #skew
#     feat[31] = np.arcsin(skew(y)/averageLength(x,y,z))
#     #kurtosis
#     feat[32] = np.arcsin(kurtosis(y)/averageLength(x,y,z))
    
    return feat  

In [116]:
# Tạo mảng features 

train_features = []
test_features = []
for action in train_data:
    feat = featuresFromBuffer(action)
    train_features.append(feat)  

for action in test_data:
    feat = featuresFromBuffer(action)
    test_features.append(feat)

len(train_features)
len(test_features)

6962

4639

In [117]:
#lower accuracy (nan value -> 0)
np.where(np.isnan(train_features)) 
train_features = np.nan_to_num(train_features)
np.where(np.isnan(test_features))
test_features = np.nan_to_num(test_features)      

len(train_features)
len(train_label)

(array([], dtype=int64), array([], dtype=int64))

(array([], dtype=int64), array([], dtype=int64))

6962

6962

In [118]:
#Classifier 

from sklearn.feature_selection import SelectFromModel
print("Gradient Boosting Decision Tree:")
from sklearn.ensemble import GradientBoostingClassifier
clf1 = GradientBoostingClassifier(learning_rate=0.05,max_depth=3,n_estimators=100).fit(train_features, train_label)
#format: pass score in {:.3f}
print('Accuracy of GBDT classifier on training set: {:.3f}'
     .format(clf1.score(train_features, train_label)))
print('Accuracy of GBDT classifier on test set: {:.3f}'
     .format(clf1.score(test_features, test_label)))

print("\n\nDecision Tree:") 
from sklearn.tree import DecisionTreeClassifier
from adspy_shared_utilities import plot_decision_tree 

clf2 = DecisionTreeClassifier(max_depth=4).fit(train_features, train_label)

print('Accuracy of Decision Tree classifier on training set: {:.3f}'.format(clf2.score(train_features, train_label)))
print('Accuracy of Decision Tree classifier on test set: {:.3f}'
.format(clf2.score(test_features, test_label)))


print("\n\nSVM:")
clf3 = SVC(C=100, gamma='scale').fit(train_features, train_label)
print("Accuracy on training set: {:.2f}".format(clf3.score(train_features, train_label)))
print("Accuracy on test set: {:.2f}".format(clf3.score(test_features, test_label)))


print('\n\nRandom Forests: ')
from sklearn.ensemble import RandomForestClassifier
clf4 = RandomForestClassifier(n_estimators=200, random_state=0).fit(train_features, train_label)

print('Accuracy of RF classifier on training set: {:.3f}'
     .format(clf4.score(train_features, train_label)))
print('Accuracy of RF classifier on test set: {:.3f}'
     .format(clf4.score(test_features, test_label)))


print('\n\nKNeighbor: ')
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 5, weights = 'distance').fit(train_features, train_label)
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(train_features, train_label)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(test_features, test_label)))


Gradient Boosting Decision Tree:
Accuracy of GBDT classifier on training set: 0.876
Accuracy of GBDT classifier on test set: 0.621


Decision Tree:
Accuracy of Decision Tree classifier on training set: 0.824
Accuracy of Decision Tree classifier on test set: 0.607


SVM:
Accuracy on training set: 0.84
Accuracy on test set: 0.63


Random Forests: 
Accuracy of RF classifier on training set: 1.000
Accuracy of RF classifier on test set: 0.641


KNeighbor: 
Accuracy of K-NN classifier on training set: 1.00
Accuracy of K-NN classifier on test set: 0.58
